In [5]:
import pandas as pd
import numpy as np

In [36]:
df = pd.read_parquet("steps400w2m_2.gzip")
df = df[df.tag.str.contains(3, regex=False) == True]
df = df.iloc[:500000]

In [9]:
df.reset_index(drop=True)

,samples,tag
0,"In a heavy 2-quart saucepan, mix brown sugar, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Place chipped beef on bottom of baking dish. P...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"In a slow cooker, combine all ingredients. Cov...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, ..."
3,Boil and debone chicken. Put bite size pieces ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Combine first 4 ingredients and press in 13 x ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
499995,"Saute the green peppers, celery, onion and pim...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
499996,Measure 3 cups of flour into large bowl; answe...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
499997,Boil together for 1 minute the last 5 ingredie...,"[0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
499998,Put fish fillets in a bowl with an airtight li...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [50]:
import random
# new_traindata = pd.read_csv("steps400w100000t.csv")
# new_traindata = train_data[train_data['tag'].str.contains('3') == True]
# new_traindata = new_traindata.reset_index(drop=True)
# new_traindata['tag'] = new_traindata['tag'].apply(ast.literal_eval)
# new_traindata = new_traindata.sample(frac=1)
traindata_t, traindata_f = np.split(df.sample(frac=1,random_state=333),
                                                      [int(.5*len(df))])
ttag = [[1,0]]*len(traindata_t)
ftag = [[0,1]]*len(traindata_f)
def randhalf():
    return random.randint(0,20)*0.2

sents = []
targets = []
count = 0
progess = 0
for row in traindata_f.iterrows():
    progess += 1
    print(progess, end='\r')
    randomized = False
    sent = row[1]['samples'].split()
    sent_len = len(row[1]['tag'])
    # target = [0]*sent_len
    bracket = False
    while randomized == False:
        for i in range(sent_len):
        # while randomized == False:
            if row[1]['tag'][i] == 3:
                if sent[i][0] == '(':
                    target = float(sent[i][1:])
                    bracket = True
                elif 'time' in sent[i]:
                    continue
                else:
                    target = float(sent[i])
                if random.randint(1,4) == 1:
                    randomized = True
                    if random.randint(1,5) == 1:
                        ran_tar = str(round(target*10,2))
                        sent[i] = '('+ran_tar if bracket else ran_tar
                    else:
                        ran_tar = str(round(target*randhalf(),2))
                        sent[i] = '('+ran_tar if bracket else ran_tar
            bracket = False
    # if 
    sents.append(sent)
    # targets.append(target)
    count += 1

joined_sents = [" ".join(s) for s in sents]

fdf = pd.DataFrame()
fdf['sentences'] = joined_sents
fdf['target'] = ftag

tdf = pd.DataFrame()
tdf['sentences'] = traindata_t['samples']
tdf['target'] = ttag
processed_training_df = pd.concat([fdf,tdf]).reset_index(drop=True)

In [54]:
# processed_training_df
processed_training_df.iloc[123154][0]
# df.iloc[439021][0]

'Chop onions; saute in oil until soft. Crush garlic; add to onions. Chop or puree tomatoes; add to onions and garlic. Add the salt, pepper and curry powder to the sauce. Simmer for 16.0 minutes. You may need to add a little water to the sauce. Pull all skin and fat from chicken. Add chicken to sauce and simmer for 135.0 minutes, stirring occasionally. Serve with pita bread, rice and salad sprinkled with lemon.'

In [56]:
processed_training_df.to_parquet("500k_dist_fsb.gzip", compression="gzip")

In [10]:
somelist = []
for row in df.iterrows():
    somelist.append(row)
somelist

[(0,
  samples    In a heavy 2-quart saucepan, mix brown sugar, ...
  tag        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
  Name: 0, dtype: object),
 (1,
  samples    Place chipped beef on bottom of baking dish. P...
  tag        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
  Name: 1, dtype: object),
 (2,
  samples    In a slow cooker, combine all ingredients. Cov...
  tag        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, ...
  Name: 2, dtype: object),
 (3,
  samples    Boil and debone chicken. Put bite size pieces ...
  tag        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
  Name: 3, dtype: object),
 (4,
  samples    Combine first 4 ingredients and press in 13 x ...
  tag        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
  Name: 4, dtype: object),
 (5,
  samples    Wash potatoes; prick several times with a fork...
  tag        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
  Name: 5, dtype: object),
 (7,
  samples    Mix together both cans of corn, cr

In [19]:
import random

# original_time is the original cooking time in the recipe step
original_time = 30  # Replace this with the actual value

# Choose a standard deviation (you can experiment with different values)
std_dev = 5  # Replace this with the chosen standard deviation

# Use a normal distribution to generate a random error
error = random.gauss(30, std_dev)

# Apply the error to the original time to get the new, erroneous cooking time
erroneous_time = original_time + error

In [20]:
error

30.405997054450093

In [34]:
randlist = []
for i in range(100):
    randlist.append(random.randint(1,4))
randlist.count(1)

23

In [57]:
df1 = pd.read_parquet("10w_for_fsb.gzip")
df2 = pd.read_parquet("500k_dist_fsb.gzip")

In [58]:
df1

,sentences,target
0,Rinse mushrooms and pat dry. Remove stems; cho...,"[0, 1]"
1,Chop artichoke hearts; mix with other ingredie...,"[0, 1]"
2,"Cream shortening; gradually add sugar, beating...","[0, 1]"
3,*Use water if fresh peaches.,"[0, 1]"
4,"Mix pretzels, 3 tablespoons sugar and margarin...","[0, 1]"
...,...,...
6533,Bake shell in very hot oven (450 degrees) for ...,"[1, 0]"
96434,Preheat oven to 350 degrees. Grease a 13 x 9-i...,"[1, 0]"
87966,"`Boil sugar, water and salt to hard boil stage...","[1, 0]"
29534,Brown chops slowly on both sides in oil. Remov...,"[1, 0]"


In [59]:
df2

,sentences,target
0,"Put tomatoes, peppers, onions and garlic in fo...","[0, 1]"
1,"Combine oats, coconut and nuts in large pan (1...","[0, 1]"
2,Crumble crackers; pour in sweet milk and let s...,"[0, 1]"
3,Pour 8 ounces of pop in pan and sprinkle 1 pac...,"[0, 1]"
4,Cream the cream cheese and sugar. Set aside. P...,"[0, 1]"
...,...,...
499995,Combine all ingredients and bake at 350 degree...,"[1, 0]"
499996,"Place onions, oleo, flour and bouillon cubes i...","[1, 0]"
499997,Preheat oven to 325 degrees. In a large mixing...,"[1, 0]"
499998,"Beat egg whites until stiff. Fold in sugar, cr...","[1, 0]"
